In [2]:
# imports
import sentiment
import importlib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from datetime import timedelta


[nltk_data] Downloading package punkt to /Users/bigdelle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
importlib.reload(sentiment)

[nltk_data] Downloading package punkt to /Users/bigdelle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'sentiment' from '/Users/bigdelle/dev/PC_RinehiML_brb227_gwr47_igp4_jkc97_rjc398/models/sentiment.py'>

In [3]:
valid_df, invalid_tickers = sentiment.process_with_yfinance("../data/earning_calls", max_files=800)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

Fetching stock data:  46%|████▌     | 365/800 [00:46<00:56,  7.70it/s]

Error fetching price changes for GL: '2021-04-24'


Fetching stock data: 100%|██████████| 800/800 [01:41<00:00,  7.86it/s]
/opt/anaconda3/envs/cs4701/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing documents: 100%|██████████| 800/800 [51:40<00:00,  3.88s/document]  


In [4]:
valid_df

,ticker,finbert_sentiment,sentiment_intensity,mean_neg_prob,mean_neu_prob,mean_pos_prob,call_date,change_1d,change_2d,change_5d
0,BMY,positive,0.322149,0.033255,0.611342,0.355404,2021-10-27,1.132948,3.381126,3.876781
1,BMY,positive,0.322149,0.033255,0.611342,0.355404,2021-07-28,-0.843907,-1.251279,-1.033040
2,BMY,positive,0.322149,0.033255,0.611342,0.355404,2022-02-04,NaN,NaN,3.308214
3,BMY,positive,0.322149,0.033255,0.611342,0.355404,2022-07-27,-0.147044,-1.376808,-1.590678
4,QCOM,positive,0.356306,0.031974,0.579747,0.388280,2022-02-02,-4.835257,-4.638667,-6.965984
...,...,...,...,...,...,...,...,...,...,...
1949,CTLT,positive,0.309078,0.069354,0.552213,0.378432,2022-02-01,-0.356571,-3.302990,NaN
1950,CTLT,positive,0.309078,0.069354,0.552213,0.378432,2021-05-04,2.879230,2.589372,NaN
1951,CTLT,positive,0.309078,0.069354,0.552213,0.378432,2020-08-31,-1.967567,-3.308105,NaN
1952,WRB,neutral,0.117701,0.095298,0.691702,0.212999,2021-04-20,-0.136373,-0.260346,NaN


In [5]:
df = valid_df.copy()
neg_cutoff = -1.0
pos_cutoff = 1.0

df['actual_label'] = np.where(
    df['change_5d'] < neg_cutoff, 'negative',
    np.where(df['change_5d'] > pos_cutoff, 'positive', 'neutral')
)

df = df.dropna(subset=['change_5d'])

actual = df['actual_label'].values

low_conf_neg = -0.6 
high_conf_pos = 0.6 

threshold_candidates = np.linspace(0, 1, 51)
best_t1, best_t2 = None, None
best_score = -np.inf

for t1 in threshold_candidates:
    for t2 in threshold_candidates:
        if t2 <= t1:
            continue
        
        conditions = [
            (df['sentiment_intensity'] <= low_conf_neg),
            (df['sentiment_intensity'] >= high_conf_pos),
            (df['sentiment_intensity'] > low_conf_neg) & (df['sentiment_intensity'] < t1),
            (df['sentiment_intensity'] >= t1) & (df['sentiment_intensity'] < t2),
            (df['sentiment_intensity'] >= t2) & (df['sentiment_intensity'] < high_conf_pos)
        ]
        choices = ['negative', 'positive', 'negative', 'neutral', 'positive']
        df['pred_label'] = np.select(conditions, choices, default='neutral')

        predicted = df['pred_label'].values

        acc = accuracy_score(actual, predicted)
        f1 = f1_score(actual, predicted, average='macro')

        metric_val = f1 
        if metric_val > best_score:
            best_score = metric_val
            best_t1 = t1
            best_t2 = t2

print(f"Best thresholds: t1={best_t1:.2f}, t2={best_t2:.2f} with macro-F1={best_score:.4f}")

df['pred_label'] = np.select(
    [
        (df['sentiment_intensity'] <= low_conf_neg),
        (df['sentiment_intensity'] >= high_conf_pos),
        (df['sentiment_intensity'] > low_conf_neg) & (df['sentiment_intensity'] < best_t1),
        (df['sentiment_intensity'] >= best_t1) & (df['sentiment_intensity'] < best_t2),
        (df['sentiment_intensity'] >= best_t2) & (df['sentiment_intensity'] < high_conf_pos)
    ],
    ['negative', 'positive', 'negative', 'neutral', 'positive'],
    default='neutral'
)

print(df[['sentiment_intensity', 'actual_label', 'pred_label']].head())

Best thresholds: t1=0.20, t2=0.26 with macro-F1=0.3742
   sentiment_intensity actual_label pred_label
0             0.322149     positive   positive
1             0.322149     negative   positive
2             0.322149     positive   positive
3             0.322149     negative   positive
4             0.356306     negative   positive


In [52]:
decile_analysis = sentiment.compute_decile_analysis(valid_df)

/Users/bigdelle/dev/PC_RinehiML_brb227_gwr47_igp4_jkc97_rjc398/models/sentiment.py:372: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['sentiment_range'] = pd.cut(valid_df['sentiment_intensity'], bins=bins, labels=labels, include_lowest=True)
/Users/bigdelle/dev/PC_RinehiML_brb227_gwr47_igp4_jkc97_rjc398/models/sentiment.py:373: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  range_analysis = valid_df.groupby('sentiment_range')[['change_1d', 'change_2d', 'change_5d']].mean().reset_index()


In [53]:
decile_analysis = decile_analysis.dropna()
decile_analysis

,sentiment_range,change_1d,change_2d,change_5d
20,0.025,-1.044501,-0.002775,0.081730
21,0.075,-1.701733,-3.139651,-3.383665
22,0.125,-2.039463,-2.557501,-2.476337
23,0.175,0.262512,0.081532,0.155779
24,0.225,0.248897,0.324122,0.637243
25,0.275,1.134444,1.524708,1.266767
26,0.325,1.397436,1.455468,1.030575
27,0.375,1.875504,2.120428,1.600460
28,0.425,0.492019,0.445362,0.145470
29,0.475,1.635353,1.984629,0.841080


In [54]:
sentiment.save_decile_analysis(decile_analysis, 'decile_analysis.pkl')

Decile analysis saved to decile_analysis.pkl


In [62]:
# Example sentiment intensity score
sentiment_intensity = 0.9

# Classify and explain
explanation = sentiment.classify_and_explain(sentiment_intensity, 'decile_analysis.pkl')
print(explanation)

Decile analysis loaded from decile_analysis.pkl
This stock is classified this way because stocks with sentiment intensity closest to 0.90 typically see an average change of 2.52% after 1 day, 2.75% after 2 days, and 3.05% after 5 days.


In [5]:
ventile_analysis = sentiment.load_decile_analysis()
ventile_analysis

Decile analysis loaded from decile_analysis.pkl


,sentiment_range,change_1d,change_2d,change_5d
0,0.025,-1.044501,-0.002775,0.081730
1,0.075,-1.701733,-3.139651,-3.383665
2,0.125,-2.039463,-2.557501,-2.476337
3,0.175,0.262512,0.081532,0.155779
4,0.225,0.248897,0.324122,0.637243
5,0.275,1.134444,1.524708,1.266767
6,0.325,1.397436,1.455468,1.030575
7,0.375,1.875504,2.120428,1.600460
8,0.425,0.492019,0.445362,0.145470
9,0.475,1.635353,1.984629,0.841080
